# MDB/DBF Conversion Test: Subprocess vs Shell Commands

This notebook comprehensively tests MDB and DBF file conversions using two approaches:
1. **Test Case 1**: Python subprocess module
2. **Test Case 2**: Databricks %sh shell commands

Each conversion is run separately to ensure clear results and error isolation.

## Setup: Install PyForge CLI

In [ ]:
# Install PyForge CLI with MDB/DBF support
%pip install /Volumes/cortex_dev_catalog/sandbox_testing/pkgs/usa-sdandey@deloitte.com/pyforge_cli-1.0.9.dev7-py3-none-any.whl --no-cache-dir --quiet --index-url https://pypi.org/simple/ --trusted-host pypi.org

In [ ]:
# Restart Python to ensure clean import
dbutils.library.restartPython()

## Define Test Files

In [ ]:
# Define MDB and DBF test files (only Parquet format supported)
test_files = [
    # MDB files
    {
        "type": "MDB",
        "name": "access_sakila.mdb",
        "path": "/Volumes/cortex_dev_catalog/0000_santosh/volume_sandbox/sample-datasets/access/small/access_sakila.mdb",
        "format": "parquet"
    },
    {
        "type": "MDB",
        "name": "sample_dibi.mdb",
        "path": "/Volumes/cortex_dev_catalog/0000_santosh/volume_sandbox/sample-datasets/access/small/sample_dibi.mdb",
        "format": "parquet"
    },
    # DBF files
    {
        "type": "DBF",
        "name": "tl_2024_us_county.dbf",
        "path": "/Volumes/cortex_dev_catalog/0000_santosh/volume_sandbox/sample-datasets/dbf/medium/tl_2024_us_county.dbf",
        "format": "parquet"
    },
    {
        "type": "DBF",
        "name": "tl_2024_01_place.dbf",
        "path": "/Volumes/cortex_dev_catalog/0000_santosh/volume_sandbox/sample-datasets/dbf/small/tl_2024_01_place.dbf",
        "format": "parquet"
    }
]

print(f"Total test files: {len(test_files)}")
print(f"MDB files: {sum(1 for f in test_files if f['type'] == 'MDB')}")
print(f"DBF files: {sum(1 for f in test_files if f['type'] == 'DBF')}")
print(f"\nAll conversions will output to Parquet format")

## Environment Verification

In [ ]:
import subprocess
import os
import time
from datetime import datetime

print("=" * 70)
print("ENVIRONMENT VERIFICATION")
print("=" * 70)
print(f"Test Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Working Directory: {os.getcwd()}")
print(f"IS_SERVERLESS: {os.environ.get('IS_SERVERLESS', 'Not set')}")

# Check PyForge version
result = subprocess.run(['pyforge', '--version'], capture_output=True, text=True)
print(f"\nPyForge Version: {result.stdout.strip()}")

# Check Java version
result = subprocess.run(['java', '-version'], capture_output=True, text=True)
if result.stderr:
    # Extract version outside of f-string to avoid backslash issue
    java_version = result.stderr.split()[2].strip('"')
    print(f"Java Version: {java_version}")
else:
    print("Java Version: Not found")

## Test Case 1: Subprocess Approach

In [ ]:
# Clean up output files that would be generated from test_files
print("🧹 Cleaning up previous test output files...")

# Generate expected output file paths based on test_files variable
expected_outputs = []

for test_file in test_files:
    # Extract base name without extension (e.g., "access_sakila" from "access_sakila.mdb")
    base_name = test_file['name'].rsplit('.', 1)[0]
    
    # PyForge CLI creates outputs in current directory by default
    # Format: basename.parquet (single file) or basename_parquet/ (directory for multi-table)
    expected_outputs.extend([
        f"{base_name}.parquet",      # Single parquet file
        f"{base_name}_parquet/"      # Directory for multi-table outputs
    ])

print(f"📋 Checking for output files from {len(test_files)} test files...")
print(f"    Expected outputs: {len(expected_outputs)} files/directories")

files_removed = 0
dirs_removed = 0

# Check each expected output
for output_path in expected_outputs:
    try:
        if output_path.endswith('/'):
            # Directory path
            try:
                dbutils.fs.ls(output_path)
                # Directory exists, remove it
                dbutils.fs.rm(output_path, recurse=True)
                dirs_removed += 1
                print(f"   🗑️ Directory exists, removed: {output_path}")
            except:
                # Directory doesn't exist - this is normal
                pass
        else:
            # File path  
            try:
                dbutils.fs.ls(output_path)
                # File exists, remove it
                dbutils.fs.rm(output_path)
                files_removed += 1
                print(f"   🗑️ File exists, removed: {output_path}")
            except:
                # File doesn't exist - this is normal
                pass
                
    except Exception as e:
        print(f"   ⚠️ Error checking {output_path}: {e}")

# Summary
total_removed = files_removed + dirs_removed
if total_removed == 0:
    print("   ✅ No existing output files found - clean workspace")
else:
    print(f"\n✅ Cleanup completed: {files_removed} files + {dirs_removed} directories removed")

print("=" * 60)

## Cleanup: Remove Previous Test Files

### Test 1.1: MDB to Parquet - access_sakila.mdb (subprocess)

In [ ]:
# Test 1.1: Convert access_sakila.mdb to Parquet
test_file = test_files[0]  # access_sakila.mdb

print(f"\n{'='*60}")
print(f"Test 1.1: {test_file['type']} to {test_file['format'].upper()} (subprocess)")
print(f"File: {test_file['name']}")
print(f"{'='*60}")

start_time = time.time()

# Run conversion without verbose flag (logs go to stderr by default)
cmd = ['pyforge', 'convert', test_file['path'], '--format', test_file['format'], '--force']
result = subprocess.run(cmd, capture_output=True, text=True)

# Display results
print(f"\nCommand: {' '.join(cmd)}")
print(f"\nExecution Time: {time.time() - start_time:.2f} seconds")
print(f"Return Code: {result.returncode}")

# Show both stdout and stderr for complete information
if result.stdout:
    print("\n--- Standard Output ---")
    print(result.stdout)
    
if result.stderr:
    print("\n--- Standard Error (includes warnings/logs) ---")
    # Show first 1000 chars to avoid too much output
    print(result.stderr[:1000])
    if len(result.stderr) > 1000:
        print(f"... (truncated, {len(result.stderr) - 1000} more characters)")
    
print(f"\nStatus: {'✅ SUCCESS' if result.returncode == 0 else '❌ FAILED'}")

# Check for generated files
import glob
output_files = glob.glob("*sakila*parquet*")
if output_files:
    print(f"\nGenerated files:")
    for f in output_files:
        print(f"  - {f}")

### Test 1.2: MDB to Parquet - sample_dibi.mdb (subprocess)

In [ ]:
# Test 1.2: Convert sample_dibi.mdb to Parquet
test_file = test_files[1]  # sample_dibi.mdb

print(f"\n{'='*60}")
print(f"Test 1.2: {test_file['type']} to {test_file['format'].upper()} (subprocess)")
print(f"File: {test_file['name']}")
print(f"{'='*60}")

start_time = time.time()

# Run conversion
cmd = ['pyforge', 'convert', test_file['path'], '--format', test_file['format'], '--force']
result = subprocess.run(cmd, capture_output=True, text=True)

# Display results
print(f"\nCommand: {' '.join(cmd)}")
print(f"\nExecution Time: {time.time() - start_time:.2f} seconds")
print(f"Return Code: {result.returncode}")

if result.stdout:
    print("\n--- Standard Output ---")
    print(result.stdout)
    
if result.stderr:
    print("\n--- Standard Error (includes warnings/logs) ---")
    print(result.stderr[:1000])
    if len(result.stderr) > 1000:
        print(f"... (truncated, {len(result.stderr) - 1000} more characters)")
    
print(f"\nStatus: {'✅ SUCCESS' if result.returncode == 0 else '❌ FAILED'}")

# Check for generated files
output_files = glob.glob("*dibi*parquet*")
if output_files:
    print(f"\nGenerated files:")
    for f in output_files:
        print(f"  - {f}")

### Test 1.3: DBF to Parquet - tl_2024_us_county.dbf (subprocess)

In [ ]:
# Test 1.3: Convert tl_2024_us_county.dbf to Parquet
test_file = test_files[2]  # tl_2024_us_county.dbf

print(f"\n{'='*60}")
print(f"Test 1.3: {test_file['type']} to {test_file['format'].upper()} (subprocess)")
print(f"File: {test_file['name']}")
print(f"{'='*60}")

start_time = time.time()

# Run conversion
cmd = ['pyforge', 'convert', test_file['path'], '--format', test_file['format'], '--force']
result = subprocess.run(cmd, capture_output=True, text=True)

# Display results
print(f"\nCommand: {' '.join(cmd)}")
print(f"\nExecution Time: {time.time() - start_time:.2f} seconds")
print(f"Return Code: {result.returncode}")

if result.stdout:
    print("\n--- Standard Output ---")
    print(result.stdout)
    
if result.stderr:
    print("\n--- Standard Error (includes warnings/logs) ---")
    print(result.stderr[:1000])
    if len(result.stderr) > 1000:
        print(f"... (truncated, {len(result.stderr) - 1000} more characters)")
    
print(f"\nStatus: {'✅ SUCCESS' if result.returncode == 0 else '❌ FAILED'}")

# Check for generated files
output_files = glob.glob("*county*parquet*")
if output_files:
    print(f"\nGenerated files:")
    for f in output_files:
        print(f"  - {f}")

### Test 1.4: DBF to Parquet - tl_2024_01_place.dbf (subprocess)

In [ ]:
# Test 1.4: Convert tl_2024_01_place.dbf to Parquet
test_file = test_files[3]  # tl_2024_01_place.dbf

print(f"\n{'='*60}")
print(f"Test 1.4: {test_file['type']} to {test_file['format'].upper()} (subprocess)")
print(f"File: {test_file['name']}")
print(f"{'='*60}")

start_time = time.time()

# Run conversion
cmd = ['pyforge', 'convert', test_file['path'], '--format', test_file['format'], '--force']
result = subprocess.run(cmd, capture_output=True, text=True)

# Display results
print(f"\nCommand: {' '.join(cmd)}")
print(f"\nExecution Time: {time.time() - start_time:.2f} seconds")
print(f"Return Code: {result.returncode}")

if result.stdout:
    print("\n--- Standard Output ---")
    print(result.stdout)
    
if result.stderr:
    print("\n--- Standard Error (includes warnings/logs) ---")
    print(result.stderr[:1000])
    if len(result.stderr) > 1000:
        print(f"... (truncated, {len(result.stderr) - 1000} more characters)")
    
print(f"\nStatus: {'✅ SUCCESS' if result.returncode == 0 else '❌ FAILED'}")

# Check for generated files
output_files = glob.glob("*place*parquet*")
if output_files:
    print(f"\nGenerated files:")
    for f in output_files:
        print(f"  - {f}")

## Test Case 2: Shell Command Approach (%sh)

### Test 2.1: MDB to Parquet - access_sakila.mdb (%sh)

In [ ]:
%%sh
echo "Command: pyforge convert /Volumes/cortex_dev_catalog/0000_santosh/volume_sandbox/sample-datasets/access/small/access_sakila.mdb --format parquet --force"
echo ""
# Run without verbose flag and capture all output
pyforge convert /Volumes/cortex_dev_catalog/0000_santosh/volume_sandbox/sample-datasets/access/small/access_sakila.mdb --format parquet --force 2>&1
echo ""
echo "Exit code: $?"

### Test 2.2: MDB to Parquet - sample_dibi.mdb (%sh)

In [ ]:
%%sh
echo "Command: pyforge convert /Volumes/cortex_dev_catalog/0000_santosh/volume_sandbox/sample-datasets/access/small/sample_dibi.mdb --format parquet --force"
echo ""
# Run without verbose flag and capture all output
pyforge convert /Volumes/cortex_dev_catalog/0000_santosh/volume_sandbox/sample-datasets/access/small/sample_dibi.mdb --format parquet --force 2>&1
echo ""
echo "Exit code: $?"

### Test 2.3: DBF to Parquet - tl_2024_us_county.dbf (%sh)

In [ ]:
%%sh
echo "Command: pyforge convert /Volumes/cortex_dev_catalog/0000_santosh/volume_sandbox/sample-datasets/dbf/medium/tl_2024_us_county.dbf --format parquet --force"
echo ""
# Run without verbose flag and capture all output
pyforge convert /Volumes/cortex_dev_catalog/0000_santosh/volume_sandbox/sample-datasets/dbf/medium/tl_2024_us_county.dbf --format parquet --force 2>&1
echo ""
echo "Exit code: $?"

### Test 2.4: DBF to Parquet - tl_2024_01_place.dbf (%sh)

In [ ]:
%%sh
echo "Command: pyforge convert /Volumes/cortex_dev_catalog/0000_santosh/volume_sandbox/sample-datasets/dbf/small/tl_2024_01_place.dbf --format parquet --force"
echo ""
# Run without verbose flag and capture all output
pyforge convert /Volumes/cortex_dev_catalog/0000_santosh/volume_sandbox/sample-datasets/dbf/small/tl_2024_01_place.dbf --format parquet --force 2>&1
echo ""
echo "Exit code: $?"

## Results Summary

In [ ]:
# List all generated files
import glob

print("=" * 70)
print("GENERATED FILES")
print("=" * 70)

# Check for Parquet files
parquet_files = glob.glob("*.parquet") + glob.glob("*_parquet/*.parquet")
if parquet_files:
    print("\nParquet files:")
    for f in sorted(parquet_files):
        size = os.path.getsize(f) / 1024
        print(f"  📁 {f} ({size:.1f} KB)")
else:
    print("\n❌ No Parquet files found")

# Check for directories
dirs = [d for d in os.listdir('.') if os.path.isdir(d) and '_parquet' in d]
if dirs:
    print("\nOutput directories:")
    for d in sorted(dirs):
        files = len(os.listdir(d))
        total_size = sum(os.path.getsize(os.path.join(d, f)) for f in os.listdir(d)) / 1024 / 1024
        print(f"  📂 {d} ({files} files, {total_size:.1f} MB total)")
        # List files in directory
        for f in sorted(os.listdir(d))[:5]:  # Show first 5 files
            file_size = os.path.getsize(os.path.join(d, f)) / 1024
            print(f"     - {f} ({file_size:.1f} KB)")
        if len(os.listdir(d)) > 5:
            print(f"     ... and {len(os.listdir(d)) - 5} more files")
else:
    print("\n❌ No output directories found")

## Test Summary

In [ ]:
print("=" * 70)
print("TEST EXECUTION SUMMARY")
print("=" * 70)
print(f"\nEnvironment: Databricks Serverless")
print(f"PyForge Version: 1.0.9.dev7")
print(f"Test Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print("\n📊 Test Coverage:")
print("  - MDB → Parquet (subprocess & %sh)")
print("  - DBF → Parquet (subprocess & %sh)")
print("  - Total: 4 file types, 2 execution methods")

print("\n🔍 Key Observations:")
print("  - Both subprocess and %sh methods should produce identical results")
print("  - Subprocess captures both stdout and stderr for debugging")
print("  - %sh shows output directly but mixing stdout/stderr")
print("  - Standard logging information provided without verbose flag")
print("  - MDB files use subprocess backend in Serverless (JPype limitation)")
print("  - DBF files use native Python reader (no Java required)")

print("\n📝 Logging Notes:")
print("  - Warning messages (like 'Table film_text is empty') are normal")
print("  - Progress is shown on stderr (not stdout) by design")
print("  - Use 2>&1 in %sh to capture all output")

print("\n✅ All tests demonstrate PyForge CLI functionality in Databricks Serverless")
print("=" * 70)